### Install Required Packages

In [1]:
!pip install hazm
!pip install tokenizers

     |████████████████████████████████| 316 kB 9.7 MB/s 
     |████████████████████████████████| 233 kB 42.2 MB/s 
     |████████████████████████████████| 1.4 MB 32.3 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=56de3de1eaa607319e61fd3b5b026d648c399bde207f56cf6e1d55b8b5152b89
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155068 sha256=8e45a6b87b2958ce95afa20e20b0f6300a462dffd51af90e98ce2920d382bbde
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 6.8 MB 8.8 MB/s 


### Import Required Packages

In [2]:
import glob
from hazm import *
import codecs
import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import yaml
import glob
import matplotlib.pyplot as plt
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
import pandas as pd

### Read data

In [6]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3GUGSQ27nG2LZzx6iOGyOpXGSH5HAED' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-3GUGSQ27nG2LZzx6iOGyOpXGSH5HAED" -O poems.zip && rm -rf /tmp/cookies.txt
!unzip poems.zip
!mkdir Models

--2022-02-18 20:12:08--  https://docs.google.com/uc?export=download&confirm=&id=1-3GUGSQ27nG2LZzx6iOGyOpXGSH5HAED
Resolving docs.google.com (docs.google.com)... 74.125.142.100, 74.125.142.102, 74.125.142.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nktnd49ftgshj4sfb5rq68vc8b92hc79/1645215075000/07093824109256912489/*/1-3GUGSQ27nG2LZzx6iOGyOpXGSH5HAED?e=download [following]
--2022-02-18 20:12:09--  https://doc-08-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nktnd49ftgshj4sfb5rq68vc8b92hc79/1645215075000/07093824109256912489/*/1-3GUGSQ27nG2LZzx6iOGyOpXGSH5HAED?e=download
Resolving doc-08-84-docs.googleusercontent.com (doc-08-84-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-08-84-docs.googleusercontent.com (doc-08-84-

In [5]:
poets = glob.glob('./*.txt')
poems = []
original_poems = []

for poem_file in poets:
    with open(poem_file, encoding='utf-8', mode='r') as fp:
        line = fp.readline()
        cnt = 1
        box = ''
        while line:
            if line.strip() != '':
                box = box + '\n' + line.strip()
                if cnt % 2 == 0:
                    original_poems.append(box)
                    tokens = word_tokenize(box)
                    refined_tokens = []
                    for token in tokens:
                        refined_tokens.append(token)
                    poems.append(' '.join(refined_tokens))
                    box = ''
                cnt += 1
            line = fp.readline()

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(poems)]

## Doc2Vec

### PV-DM, Vector_size = 100

In [ ]:
max_epochs = 20
vec_size = 100
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in tqdm.tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 0


  5%|▌         | 1/20 [06:45<2:08:27, 405.67s/it]

iteration 1


 10%|█         | 2/20 [13:24<2:00:31, 401.74s/it]

...

 90%|█████████ | 18/20 [1:58:39<13:12, 396.08s/it]

iteration 18


 95%|█████████▌| 19/20 [2:05:16<06:36, 396.37s/it]

iteration 19


100%|██████████| 20/20 [2:11:56<00:00, 395.84s/it]


In [ ]:
model.save('./Models/PV_DM_vec100')

### PV-DM, Vector_size = 300

In [ ]:
max_epochs = 20
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in tqdm.tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()
100%|██████████| 20/20 [2:18:36<00:00, 415.83s/it]


In [ ]:
model.save('./Models/PV_DM_vec300')

### PV-DBOW, Vector_size = 100

In [ ]:
max_epochs = 20
vec_size = 100
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm = 0)
  
model.build_vocab(tagged_data)

for epoch in tqdm.tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()
100%|██████████| 20/20 [1:12:52<00:00, 218.61s/it]


In [ ]:
model.save('./Models/PV_DBOW_vec100')

### PV-DBOW, Vector_size = 300

In [ ]:
max_epochs = 20
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm = 0)
  
model.build_vocab(tagged_data)

for epoch in tqdm.tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()
100%|██████████| 20/20 [1:20:21<00:00, 241.06s/it]


In [ ]:
model.save('./Models/PV_DBOW_vec300')

## Zip saved models

In [ ]:
!zip -r doc2vec.zip ./Models

  adding: Models/ (stored 0%)
  adding: Models/PV_DM_vec300.wv.vectors.npy (deflated 7%)
  adding: Models/PV_DM_vec300.docvecs.vectors_docs.npy (deflated 7%)
  adding: Models/PV_DM_vec300.trainables.syn1neg.npy (deflated 7%)
  adding: Models/PV_DM_vec300 (deflated 75%)
  adding: Models/PV_DM_vec100.wv.vectors.npy (deflated 7%)
  adding: Models/PV_DM_vec100.trainables.syn1neg.npy (deflated 7%)
  adding: Models/PV_DM_vec100.docvecs.vectors_docs.npy (deflated 7%)
  adding: Models/PV_DM_vec100 (deflated 75%)
  adding: Models/PV_DBOW_vec300.wv.vectors.npy (deflated 9%)
  adding: Models/PV_DBOW_vec300.trainables.syn1neg.npy (deflated 7%)
  adding: Models/PV_DBOW_vec300.docvecs.vectors_docs.npy (deflated 7%)
  adding: Models/PV_DBOW_vec300 (deflated 75%)
  adding: Models/PV_DBOW_vec100 (deflated 75%)
  adding: Models/PV_DBOW_vec100.wv.vectors.npy (deflated 8%)
  adding: Models/PV_DBOW_vec100.trainables.syn1neg.npy (deflated 7%)
  adding: Models/PV_DBOW_vec100.docvecs.vectors_docs.npy (deflate